In [550]:
# MODULES

import numpy as np
import matplotlib.pyplot as plt
from random import random, choice
import typing

In [ ]:
# CONSTANTS

FLOATING_DIGITS_IN_DNA = 2  # current: x.xx------------...(quite low precision)
INT_SIZE_IN_DNA = 8  # bits current:2**7 = 128 + 1bit(sign)
FLOAT_IN_DNA_LENGHT = INT_SIZE_IN_DNA / 2 + FLOATING_DIGITS_IN_DNA * 2 # XXXX XX XX

In [552]:
# CLASSES


class Perceptron:
    def __init__(
        self, weights: list[float|int], bias: float|int, activation: typing.Callable
    ):
        self.weights = np.array(weights)
        self.bias = bias
        self.activation = activation

    def __call__(self, inputs: list[int|float]) -> float:
        npinputs = np.array(inputs)
        return self.activation(sum(npinputs * self.weights)+self.bias)


class Network:
    layers: list[list[Perceptron]]
    activation: typing.Callable
    
    @property
    def shape(self) -> tuple[int, ...]:
        return tuple([len(layer) for layer in self.layers])

    def __call__(self, inputs: list[float|int]):
        for layer in self.layers:
            inputs = [perceptron(inputs) for perceptron in layer]
        return inputs

class DNA:
    def __init__(self, seq: str='') -> None:
        self.seq = seq
    def __str__(self) -> str:
        return self.seq
    def __add__(self, other: 'DNA') -> 'DNA':
        return DNA(self.seq + other.seq)
    def __getitem__(self, key):
        return DNA(self.seq[key])
    def __len__(self):
        return len(self.seq)

In [553]:
# FUNCTIONS


def complete_bin(
    binary: str, digits: int, isneg: bool | None = None, unsigned: bool = False
) -> str:
    binary = "0" * (digits - len(binary) - (1 if not isneg == None else 0)) + binary
    return (("1" if isneg else "0") if not isneg == None else "") + binary


def int2bin(n: int) -> str:
    return bin(n).split("b")[1]


bits2dna = {"00": "A", "01": "C", "10": "G", "11": "T"}

dna2bits = {"A": "00", "C": "01", "G": "10", "T": "11"}


def bin2dna(binary: str) -> DNA:
    res = DNA()
    for i in range(0, len(binary), 2):
        word = binary[i : i + 2]
        res += DNA(bits2dna[word])
    return res


def float2dna(num: int | float) -> DNA:
    num = round(float(num), 2)

    integer = int(str(num).split(".")[0])
    try:
        first, second = str(num).split(".")[1]
    except ValueError:
        first, second = str(num).split(".")[1], "0"
    first, second = int(first), int(second)

    integer_bin = int2bin(integer)
    first_bin = int2bin(first)
    second_bin = int2bin(second)

    if len(integer_bin) > 7:
        raise Exception("too big weight value")

    integer_complete = complete_bin(integer_bin, 8, num < 0)
    first_complete = complete_bin(first_bin, 4)
    second_complete = complete_bin(second_bin, 4)

    return (
        bin2dna(integer_complete) + bin2dna(first_complete) + bin2dna(second_complete)
    )


def dna2float(dna: DNA) -> float:
    integer_dna = dna[:4]
    first_dna = dna[4:6]
    second_dna = dna[6:]

    integer_bin = "".join([dna2bits[str(some)] for some in integer_dna])
    first_bin = "".join([dna2bits[str(some)] for some in first_dna])
    second_bin = "".join([dna2bits[str(some)] for some in second_dna])

    integer = int(integer_bin[1:], 2) * (-1 if int(integer_bin[0]) else 1)
    first = int(first_bin, 2)
    second = int(second_bin, 2)

    return float(f"{integer}.{first}{second}")


def perceptron2dna(perceptron: Perceptron) -> DNA:
    weights_dna = DNA()
    for weight in perceptron.weights:
        weights_dna += float2dna(weight)
    return float2dna(len(perceptron.weights)) + weights_dna + float2dna(perceptron.bias)


def dna2perceptron(dna: DNA, activation: typing.Callable) -> Perceptron:
    inputs = int(dna2float(dna[:8]))
    weights_dna = dna[8 : 8 + (inputs * 8)]
    bias_dna = dna[8 + (inputs * 8) :]

    weights = []
    for i in range(0, (inputs) * 8, 8):
        weight_dna = weights_dna[i : i + 8]
        weights.append(dna2float(weight_dna))

    return Perceptron(np.array(weights), dna2float(bias_dna), activation)


def network2dna(network: Network) -> DNA:
    network_dna = DNA()
    for shape, layer in zip(network.shape, network.layers):
        network_dna += float2dna(shape)
        network_dna += float2dna(8*2 + 8*len(layer[0].weights))
        for perceptron in layer:
            network_dna+=perceptron2dna(perceptron)
    return network_dna


def dna2network(dna: DNA) -> Network:
    pass